In [1]:
from google import colab

colab.drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
data_path = '/content/drive/Shareddrives/JobCare/Jobcare_data/'
submit_path = '/content/drive/Shareddrives/JobCare/submit/'
SEED = 42

In [3]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

In [4]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 60 kB/s 


In [5]:
# Optuna Libraries
!pip install optuna
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

# LGBM Regressor
from lightgbm import LGBMRegressor

# train_test_split
from sklearn.model_selection import train_test_split

# Evaluation Score
from sklearn.metrics import mean_squared_error

     |████████████████████████████████| 308 kB 27.9 MB/s 
     |████████████████████████████████| 209 kB 78.0 MB/s 
     |████████████████████████████████| 80 kB 12.5 MB/s 
     |████████████████████████████████| 75 kB 6.0 MB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
     |████████████████████████████████| 112 kB 80.3 MB/s 
     |████████████████████████████████| 149 kB 73.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=21ea6f67a608ef089f9d8e009efda656c84389989a977fac87b35badf8e912c9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [6]:
train_data = pd.read_csv(f'{data_path}train.csv')
test_data = pd.read_csv(f'{data_path}test.csv')

code_d = pd.read_csv(f'{data_path}속성_D_코드.csv')
code_h = pd.read_csv(f'{data_path}속성_H_코드.csv')
code_l = pd.read_csv(f'{data_path}속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [7]:
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m","attribute_d_l"]
code_h.columns= ["attribute_h","attribute_h_p","attribute_h_l"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [8]:
#데이터 merge
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    df_code.columns.values[0] = col
    return pd.merge(df,df_code,how="left",on=col)

In [9]:
def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True, cols_merge:List[Tuple[str,pd.DataFrame]] = []  , cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()

    y_data = None
    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col, df_code in cols_merge:
        df = merge_codes(df,df_code,col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

    df = df.drop(columns=cols_drop)
    return (df , y_data)

In [10]:
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_l" , "contents_attribute_d_attribute_d_l"),

    ("person_prefer_h_1_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_2_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_3_attribute_h_p" , "contents_attribute_h_attribute_h_p"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]

In [11]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
x_train.shape , y_train.shape , x_test.shape

((501951, 68), (501951,), (46404, 68))

In [ ]:
# random sampler
sampler = TPESampler(seed=10)
from catboost import Pool,CatBoostClassifier

# define function
def objective(trial):
    test_pred = pd.Series([0 for x in range(len(x_test))], index=x_test.index)
    kf = KFold(n_splits=10,shuffle = True, random_state=2014)
    for train_index, valid_index in kf.split(x_train):
      train_X, train_y = x_train.iloc[train_index], y_train[train_index]
      valid_X, valid_y = x_train.iloc[valid_index], y_train[valid_index]
      

      cbrm_param = {
          "eval_metric" : 'AUC',
          "iterations" : 5000,
          "metric_period" : 5000,
          "early_stopping_rounds": 1000,
          "task_type" : 'GPU',
          "grow_policy" : 'Depthwise',
          "depth" : trial.suggest_int("depth", 4, 12),
          'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
          "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",20,50),
          "random_seed": 2014,

        }
      

      # Generate model
      model_cbrm = CatBoostClassifier(**cbrm_param)
      model_cbrm = model_cbrm.fit(train_X, train_y, eval_set=(valid_X, valid_y))

      # 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
      MSE = mean_squared_error(valid_y, model_cbrm.predict(valid_X))
      return MSE


optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=50)



[I 2022-01-17 01:49:24,571] A new study created in memory with name: no-name-8a7cf5f5-2201-4d2b-bc9f-82b20c7b97ff


0:	learn: 0.6495224	test: 0.6358535	best: 0.6358535 (0)	total: 108ms	remaining: 8m 59s
4999:	learn: 0.9302554	test: 0.6984776	best: 0.6984776 (4999)	total: 4m 11s	remaining: 0us
bestTest = 0.698477596
bestIteration = 4999


[I 2022-01-17 01:54:00,063] Trial 0 finished with value: 0.3560443063192286 and parameters: {'depth': 10, 'learning_rate': 0.010731320001845925, 'l2_leaf_reg': 39.009447047788264}. Best is trial 0 with value: 0.3560443063192286.


0:	learn: 0.6496345	test: 0.6359600	best: 0.6359600 (0)	total: 103ms	remaining: 8m 36s
bestTest = 0.6982039213
bestIteration = 1624
Shrink model to first 1625 iterations.


[I 2022-01-17 01:56:25,037] Trial 1 finished with value: 0.3562634472866364 and parameters: {'depth': 10, 'learning_rate': 0.054494830174039256, 'l2_leaf_reg': 26.74389936592543}. Best is trial 0 with value: 0.3560443063192286.


...

0:	learn: 0.6582738	test: 0.6337798	best: 0.6337798 (0)	total: 165ms	remaining: 13m 45s
4999:	learn: 0.9951435	test: 0.7015440	best: 0.7015642 (4430)	total: 6m 45s	remaining: 0us
bestTest = 0.7015641928
bestIteration = 4430
Shrink model to first 4431 iterations.


[I 2022-01-17 03:13:26,842] Trial 16 finished with value: 0.3551278986373416 and parameters: {'depth': 11, 'learning_rate': 0.017767503088258734, 'l2_leaf_reg': 33.29140311305197}. Best is trial 12 with value: 0.35297633277552.


0:	learn: 0.6307722	test: 0.6270919	best: 0.6270919 (0)	total: 36.8ms	remaining: 3m 3s
4999:	learn: 0.8320373	test: 0.6930404	best: 0.6930451 (4995)	total: 1m 35s	remaining: 0us
bestTest = 0.6930451393
bestIteration = 4995
Shrink model to first 4996 iterations.


[I 2022-01-17 03:15:10,877] Trial 17 finished with value: 0.36110447047573513 and parameters: {'depth': 7, 'learning_rate': 0.02564532014603323, 'l2_leaf_reg': 29.348313682794718}. Best is trial 12 with value: 0.35297633277552.


0:	learn: 0.6580537	test: 0.6336542	best: 0.6336542 (0)	total: 166ms	remaining: 13m 50s


In [ ]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print('Best Trial: score {},\nparams {}'.format(cbrm_trial.value, cbrm_trial_params))

In [ ]:
cbrm_trial_params

In [ ]:
data1 = catboost_modeling(x_train, y_train, x_test, 'Depthwise', 10, 0.02423, 20.35, 2014, 2)

In [ ]:
datas = pd.read_csv("/content/drive/Shareddrives/JobCare/submit/CatBoost_Depthwise_10.csv")
data_fin = pd.DataFrame(datas*2)
data_fin.columns = datas.columns
data_fin

In [ ]:
data4 = catboost_modeling(x_train, y_train, x_test, 'Lossguide', 16, 0.01213, 5.027, 2022, 2)

In [ ]:
# 최종 모델 앙상블
sample_submission = pd.read_csv(f'{data_path}sample_submission.csv')
data_final = pd.DataFrame(data1)
data_final.columns = sample_submission.columns
data_final.to_csv(f"{submit_path}final_data1_only.csv", index=True)
data_final

In [ ]:
'''# 최종 모델 앙상블
sample_submission = pd.read_csv(f'{data_path}sample_submission.csv')
data_final = pd.DataFrame((data1+data2)/2 *1/3 + (data3+data4)/2 *2/3)
data_final.columns = sample_submission.columns
data_final.to_csv('data_final.csv', index =True)
data_final'''

In [ ]:
y_pred=clf.predict(x_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

## sklearn 홈페이지 참조

In [ ]:
import pandas as pd 

jobs = pd.read_csv('/content/drive/Shareddrives/JobCare/Jobcare_data/train.csv', sep=',')

In [ ]:
print(jobs.columns)
print(jobs.id[0:5])

In [ ]:
from sklearn.model_selection import train_test_split

X=jobs[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'contents_rn']]  # Features ('id','contents_open_dt' 제외)
y=jobs['target']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import pandas as pd
jobs.newcol = []
for i in range(len(jobs.columns)):
  if i != 0 and i <= 32:
    jobs.newcol.append(jobs.columns[i])
feature_imp = pd.Series(clf.feature_importances_,index=jobs.newcol).sort_values(ascending=False)
feature_imp

In [ ]:
print(feature_imp.index[:16])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

그래서 위 요소들 중에 가장 importance 가 낮은것 제외 상위 16개만 학습하면?

In [ ]:
from sklearn.model_selection import train_test_split
# Split dataset into features and labels
X=jobs[['contents_rn', 'person_rn', 'contents_attribute_l',
       'contents_attribute_d', 'person_prefer_d_2', 'person_prefer_d_1',
       'contents_attribute_h', 'person_prefer_d_3', 'person_prefer_h_2',
       'person_prefer_h_3', 'person_prefer_h_1', 'person_attribute_a_1',
       'person_prefer_e', 'contents_attribute_e', 'person_attribute_b',
       'contents_attribute_m']]  # Removed feature "sepal length"
y=jobs['target']                                       
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=5) 



In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

# prediction on test set
y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))